# Análise de Sentimento

Introdução

A análise de sentimento é uma das áreas do Processamento de Linguagem Natural utilizada para compreender sobre a opinião de clientes, usuários e pacientes. Podemos utilizar nas redes sociais, coletando todas as publicações sobre determinada hashtag e realizar uma análise de sentimento para descobrir se os usuários concordam ou não sobre o assunto a qual a hashtag está atrelada. 

Ao coletar as publicações sobre um grande período de tempo, esta análise permite que se encontrem os momentos na qual houve uma grande aceitação e buscar repetir ou atualizar o evento na qual permitiu que o assunto tivesse uma grande concordância com o público. Por exemplo: uma empresa pode buscar quais foram as publicações sobre um produto X e, a partir das publicações negativas, relançar o produto X novamente mas sobre outra perspectiva ou lançar um produto Y corrigindo os erros do produto anterior.
A análise de sentimento é uma ferramenta que auxilia em muito a compreender o público de nosso interesse. Aprenderemos a seguir como fazer essa análise.

Usaremos os dados dos comentários publicados pelos usuários do site IMDB traduzidos por Luís Fred e disponibilizado para download pelo Kaggle.

In [1]:
import numpy as np
import pandas as pd

import spacy

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC


In [2]:
imdb = pd.read_csv('imdb-reviews-pt-br.csv')
imdb.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


Faremos um pequeno teste para validar o nosso pré-processamento que usaremos para preparar o conjunto de dados.


In [3]:
nlp = spacy.load('pt_core_news_lg')

texto = imdb.text_pt[0]

print('--------------')
print('Texto original')
print(texto)

doc = nlp(texto)
tokens_filtrado = [token for token in doc if ((not token.is_stop) & (not token.is_punct))]

print('--------------')
print('Remoção de stopwords e pontuação')
print(tokens_filtrado)

lemmas = [token.lemma_.lower().strip() for token in tokens_filtrado]

print('--------------')
print('Lemmatization')
print(lemmas)

--------------
Texto original
Mais uma vez, o Sr. Costner arrumou um filme por muito mais tempo do que o necessário. Além das terríveis seqüências de resgate no mar, das quais há muito poucas, eu simplesmente não me importei com nenhum dos personagens. A maioria de nós tem fantasmas no armário, e o personagem Costers é realizado logo no início, e depois esquecido até muito mais tarde, quando eu não me importava. O personagem com o qual deveríamos nos importar é muito arrogante e superconfiante, Ashton Kutcher. O problema é que ele sai como um garoto que pensa que é melhor do que qualquer outra pessoa ao seu redor e não mostra sinais de um armário desordenado. Seu único obstáculo parece estar vencendo Costner. Finalmente, quando estamos bem além do meio do caminho, Costner nos conta sobre os fantasmas dos Kutchers. Somos informados de por que Kutcher é levado a ser o melhor sem pressentimentos ou presságios anteriores. Nenhuma mágica aqui, era tudo que eu podia fazer para não desligar u

Iremos, a partir de agora, criar as funções que usaremos para montar um pipeline para realizar o pré-processamento, ajuste do modelo e a predição.

In [4]:
# Criamos uma classe para gerenciar X e y
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    def fit(self, X, y=None, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}

# Esta funcao remove espacos em branco no inicio e
# no fim do texto e converte todo o texto em letras
# minusculas
def clean_text(texto):     
    return texto.strip().lower()

# Esta funcao remove todas as stopwords e pontuacoes
def tokenizer(texto):
    doc = nlp(texto)
    tokens = [token for token in doc if ((not token.is_stop) & (not token.is_punct))]
    tokens = [token.lemma_.lower().strip() for token in tokens]
    return tokens

# Criamos um objeto CountVectorizer para vetorizar cada
# texto
vectorizer = CountVectorizer(tokenizer = tokenizer, ngram_range=(1,1)) 

# Criamos um modelo SVM
classifier = SVC()

**CountVectorizer:** Esta função é usada para transformar o texto em vetores de valores numéricos. Estes vetores podem ser acessados pelo modelo mais rapidamente do que o texto.

O modelo levará alguns minutos para treinar, não se preocupe. Um ponto a levar em consideração ao trabalhar com textos é que com poucos textos será exigido uma grande quantidade de processamento e memória.

In [5]:
# Separando em X e y
X = imdb.text_pt
y = imdb.sentiment

# Separando em teste e treino
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# Construindo uma pipeline
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', classifier)])

pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)

print(round(accuracy_score(y_test, y_pred), 3))
print(round(precision_score(y_test, y_pred, average='macro'), 3))
print(round(recall_score(y_test, y_pred, average='macro'), 3))

0.881
0.882
0.882
